#### Tidy Hvaler dataset

In [1]:
library(tidyr)
library(dplyr)
library(lubridate)

top20File = "Hvaler/top_20.csv"
tempFile = "Hvaler/temperature_2010_2014.csv"

#Define a class num.with.commas to probably transform string with comma to number

top20Classes = c('POSIXct', "numeric", "character")

df = read.csv(top20File, stringsAsFactors=FALSE, colClasses=top20Classes)
names(df) = c("DateTime", "Consumption", "NETTSTNR")


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [6]:
#Spread dataset and change name to make substation name arbitrary
tidyDf = df %>% spread(NETTSTNR, Consumption, fill = NA, convert = FALSE)
names(tidyDf) = c("DateTime", paste0("subs.", seq(1, ncol(tidyDf)-1)))
head(tidyDf)

#### Read temperature and join to make complete dataset

In [21]:
tempClasses = c('POSIXct', rep("numeric", 3))
tempDf = read.csv(tempFile, stringsAsFactors=FALSE, colClasses=tempClasses)
names(tempDf) = c("DateTime", "T01", "Min", "Max")
tempDf = tempDf %>% select(DateTime, T01) %>% mutate(T01 = T01 * 9/5 + 32)
completeDf = left_join(tidyDf, tempDf, by="DateTime")

In [25]:
write.csv(completeDf, "Hvaler/complete.csv", row.names=FALSE)

In [24]:
head(completeDf)

,DateTime,subs.1,subs.2,subs.3,subs.4,subs.5,subs.6,subs.7,subs.8,subs.9,⋯,subs.12,subs.13,subs.14,subs.15,subs.16,subs.17,subs.18,subs.19,subs.20,T01
1,2011-11-01 00:00:00,0,0,0,24.029999980703,12.4500000644475,29.980000289157,37.6900001205504,23.7299999035895,54.509999750182,⋯,50.4600000139326,59.3200003504753,35.5999997276813,10.5900000501424,13.4999999701977,12.4100001342595,76.4500007890165,0,43.2999998703599,51.98
2,2011-11-01 01:00:00,0,0,0,22.2100001350045,14.0400001630187,31.8599998466671,34.4000001754612,24.2699999921024,51.1100000068545,⋯,53.6099999584258,54.9099999070168,38.2599998805672,9.55999996885657,14.7299999184906,12.5500000510365,70.2399999853224,0,40.7680000793189,52.34
3,2011-11-01 02:00:00,0,0,0,25.4700001049787,14.1900000721216,31.0899999141693,28.4999999906868,22.4200000148267,56.3999998532236,⋯,51.5899997409433,57.8099996764213,36.6499996501952,10.2800000440329,15.7299999594688,10.8299998957664,76.6600002218038,0,40.2399997860193,52.16
4,2011-11-01 03:00:00,0,0,0,24.6799999475479,14.6399999819696,28.7399999182671,28.6200002226979,22.9099999964237,59.7299999501556,⋯,43.0900002699345,58.5500002130866,37.8399998731911,9.59999995678663,12.9500001128763,11.0200000666082,72.6299997810274,0,41.9019999355078,52.52
5,2011-11-01 04:00:00,0,0,0,23.1800000891089,14.4899999443442,24.9099999088794,30.2500001024455,22.7099999580532,59.7899995949119,⋯,45.8200001809746,55.3599996324629,38.9499997217208,10.6999999303371,15.319999890402,11.9300000630319,74.9999997541308,0,41.5699999462813,52.34
6,2011-11-01 05:00:00,0,0,0,22.120000006631,14.0800000838935,28.0900001078844,29.1500002034009,23.4199998304248,58.9100003521889,⋯,48.9300003126264,59.529999833554,35.3300001267344,9.5399999525398,14.7599999923259,12.4899998772889,74.3399994745851,0,47.0379999354482,52.52
